# VacationPy
----

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
# Load cities dataframe 
csv_path = '../output_data/cities.csv'
cities_df = pd.read_csv(csv_path)

# (Formatting) Convert temp. column to farenheit 
temp_col = cities_df['Max Temp']
cities_df['Max Temp (F)'] = (temp_col - 273.15) * 9/5 + 32

# Display 
cities_df

,Unnamed: 0,City,Lat,Long,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Max Temp (F)
0,0,Illoqqortoormiut,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,Bambous Virieux,-20.3428,57.7575,302.04,65.0,40.0,5.14,MU,1.612499e+09,84.002
2,2,Chifeng,42.2683,118.9636,273.58,89.0,47.0,4.93,CN,1.612499e+09,32.774
3,3,Busselton,-33.6500,115.3333,297.04,42.0,1.0,1.34,AU,1.612499e+09,75.002
4,4,Nikolskoye,59.7035,30.7861,258.71,85.0,20.0,4.00,RU,1.612499e+09,6.008
...,...,...,...,...,...,...,...,...,...,...,...
623,623,Concordia,-31.3930,-58.0209,292.04,83.0,0.0,1.60,AR,1.612499e+09,66.002
624,624,Kissamos,35.4939,23.6569,285.15,94.0,20.0,3.09,GR,1.612499e+09,53.600
625,625,Jinchengjiang,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
626,626,Babstovo,48.1167,132.4833,254.70,85.0,90.0,2.46,RU,1.612499e+09,-1.210


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
# Drop null values 
cities_df = cities_df.dropna()

# Store location and humidity data 
lat_lng = cities_df[["Lat", "Long"]].astype(float)
humidities = cities_df['Humidity'].astype(float)

In [52]:
# Configure gmaps with API key
gmaps.configure(api_key=g_key)

# Create heatmap 
gmap_map = gmaps.figure()

gmap_heat = gmaps.heatmap_layer(lat_lng, weights=humidities, dissipating=False, max_intensity=None,point_radius = 3)
gmap_map.add_layer(gmap_heat)

gmap_map

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [30]:
# Select rows that fit criteria: Humidity between 30-50%, Max. Temp > 70F, Cloudiness > 50%
criteria_df = cities_df.loc[(cities_df['Humidity'] > 30) & (cities_df['Humidity'] < 50) & (cities_df['Max Temp (F)'] > 70) & (cities_df['Cloudiness'] > 50)]

# Clean dataframe 
hotel_df = criteria_df[['City', 'Lat', 'Long', 'Max Temp (F)', 'Humidity', 'Cloudiness', 'Wind Speed', 'Country']].reset_index(drop=True)
hotel_df

,City,Lat,Long,Max Temp (F),Humidity,Cloudiness,Wind Speed,Country
0,Mataura,-46.1927,168.8643,75.434,39.0,81.0,0.82,NZ
1,Nishihara,26.1842,127.7558,73.400,37.0,75.0,4.63,JP
2,Te Anau,-45.4167,167.7167,74.588,44.0,70.0,1.16,NZ
3,Mount Gambier,-37.8333,140.7667,73.400,38.0,75.0,7.72,AU
4,Chuncheng,22.1731,111.7886,78.008,47.0,75.0,1.42,CN
5,Ratnagiri,16.9833,73.3000,81.626,38.0,92.0,2.14,IN


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [31]:
hotel_df['Hotel Name'] = ""

url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
params = {"radius": 5000,"type": "hotel", "key": g_key}

# Loop through rows and get information 
for i, row in hotel_df.iterrows():
    lat = row['Lat']
    lng = row['Long']
    city = row['City']
    
    # add location input 
    params['location'] = f'{lat},{lng}'

    # assemble url and make API request
    response = requests.get(url, params=params).json()
    
    # Print information
    print(f'Retrieving result {i} for {city}')
    
    try: 
        # get results
        results = response['results'][1]['name']
        print(f'Found: {results} in {city}')
        print('-----------------------------')
              
        # add result to column
        hotel_df.loc[i,'Hotel Name'] = results
              
    except (KeyError, IndexError):
        print(f'Missing information for {city}. Skipping..')
        print('-----------------------------')

print('Finished retrieving results')


Retrieving result 0 for Mataura
Found: Ellie's Villa in Mataura
-----------------------------
Retrieving result 1 for Nishihara
Found: JEF in Nishihara
-----------------------------
Retrieving result 2 for Te Anau
Found: Kingsgate Hotel Te Anau in Te Anau
-----------------------------
Retrieving result 3 for Mount Gambier
Found: Commodore on the Park in Mount Gambier
-----------------------------
Retrieving result 4 for Chuncheng
Found: Tianfu Garden in Chuncheng
-----------------------------
Retrieving result 5 for Ratnagiri
Found: Bajaj Allianz Life Insurance Company Limited in Ratnagiri
-----------------------------
Finished retrieving results


In [32]:
hotel_df

,City,Lat,Long,Max Temp (F),Humidity,Cloudiness,Wind Speed,Country,Hotel Name
0,Mataura,-46.1927,168.8643,75.434,39.0,81.0,0.82,NZ,Ellie's Villa
1,Nishihara,26.1842,127.7558,73.400,37.0,75.0,4.63,JP,JEF
2,Te Anau,-45.4167,167.7167,74.588,44.0,70.0,1.16,NZ,Kingsgate Hotel Te Anau
3,Mount Gambier,-37.8333,140.7667,73.400,38.0,75.0,7.72,AU,Commodore on the Park
4,Chuncheng,22.1731,111.7886,78.008,47.0,75.0,1.42,CN,Tianfu Garden
5,Ratnagiri,16.9833,73.3000,81.626,38.0,92.0,2.14,IN,Bajaj Allianz Life Insurance Company Limited


In [36]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Long"]]

In [53]:
# Add marker layer ontop of heat map
markers = gmaps.marker_layer(locations)
gmap_map.add_layer(markers)

# Display figure
gmap_map

Figure(layout=FigureLayout(height='420px'))